In [1]:
import pandas as pd
import glob, os
#from dataprep.eda import plot_correlation
#from dataprep.eda import plot
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import datetime
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
from scipy import stats
from scipy import interpolate
from sklearn.linear_model import LinearRegression
import pandas as pd
#import datacleaner
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


import random
from matplotlib import pyplot as plt
from matplotlib import colors as mpc
plt.rcParams["font.family"] = "Poppins"
from sklearn.mixture import GaussianMixture

import seaborn as sns; sns.set()
from sklearn.metrics import r2_score 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
#from gmr.utils import check_random_state
#from gmr import MVN, GMM, plot_error_ellipses
#random_state = check_random_state(0)

from IPython.display import clear_output

%load_ext autoreload
%autoreload 
%matplotlib inline


import plotly.express as px

import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
#from bioinfokit.analys import get_data, stat
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [2]:
ts = pd.read_csv('4_factor_MOD-00400/timeseries.csv')
results = pd.read_csv('4_factor_MOD-00400/results.csv')
results['localtime'] = ts['localtime']

data = pd.concat([ts,results['Factor 1'],results['Factor 2'], results['Factor 3'],results['Factor 4']],axis=1)
ts['localtime'] = pd.to_datetime(ts['localtime']) #convert timestamp column to pandas timestamp format
data = data.set_index('localtime',append=False) #setindex as local time

data.head()

,Unnamed: 0,bin0,bin1,bin2,bin3,bin4,bin5,co_diff,no_diff,no2_diff,ox_diff
localtime,,,,,,,,,,,
2024-10-01 23:59:51,0,12.422,1.372,0.183,0.031,0.038,0.018,84.3,5.7,0.6,2.7
2024-10-01 23:58:51,1,12.824,1.254,0.221,0.041,0.094,0.042,84.7,5.6,0.7,2.9
2024-10-01 23:57:51,2,16.099,1.477,0.205,0.062,0.010,0.063,93.0,6.2,1.1,2.1
2024-10-01 23:56:51,3,13.350,1.257,0.157,0.049,0.059,0.029,95.5,6.3,1.4,2.1
2024-10-01 23:55:51,4,13.767,1.220,0.256,0.051,0.083,0.020,95.8,6.2,1.8,1.9


In [3]:
data.plot(y='ws')

KeyError: 'ws'

In [ ]:
wd_rad = np.radians(np.array(data['wd'])) #deg to rad
ws = np.array(data['ws']) #m/s
Factor1 = np.array(data['Factor 1'])
Factor2 = np.array(data['Factor 2'])
Factor3 = np.array(data['Factor 3'])
Factor4 = np.array(data['Factor 4'])

In [ ]:
WD, WS = np.meshgrid(np.linspace(0, 2*np.pi, 36), np.linspace(min(ws), max(ws), 16 ))
Z1 = interpolate.griddata((wd_rad, ws), Factor1, (WD, WS), method='linear')
Z2 = interpolate.griddata((wd_rad, ws), Factor2, (WD, WS), method='linear')
Z3 = interpolate.griddata((wd_rad, ws), Factor3, (WD, WS), method='linear')
Z4 = interpolate.griddata((wd_rad, ws), Factor4, (WD, WS), method='linear')

In [ ]:
fig = plt.figure(figsize=(12,12))
ax1= fig.add_subplot(221, projection='polar')
ax1.set_title('(a) Factor 1',fontsize=14)
cmap = plt.get_cmap('hot_r')
cmap.set_under('none')
img = ax1.pcolormesh(WD, WS, Z1, cmap=cmap, vmin=0,vmax=1)
#plt.colorbar(img)
ax1.tick_params(axis='both', which='major', labelsize=14)


In [ ]:
ax2= fig.add_subplot(222, projection='polar')
ax2.set_title('(b) Factor 2',fontsize=14)
cmap = plt.get_cmap('hot_r')
cmap.set_under('none')
img = ax2.pcolormesh(WD, WS, Z2, cmap=cmap, vmin=0,vmax=1)
#plt.colorbar(img)
ax2.tick_params(axis='both', which='major', labelsize=14)

In [ ]:
ax3= fig.add_subplot(223, projection='polar')
ax3.set_title('(c) Factor 3',fontsize=14)
cmap = plt.get_cmap('hot_r')
cmap.set_under('none')
img = ax3.pcolormesh(WD, WS, Z3, cmap=cmap, vmin=0,vmax=1)
plt.colorbar(img)
ax3.tick_params(axis='both', which='major', labelsize=14)

In [ ]:
ax4= fig.add_subplot(224, projection='polar')
ax4.set_title('(c) Factor 4',fontsize=14)
cmap = plt.get_cmap('hot_r')
cmap.set_under('none')
img = ax4.pcolormesh(WD, WS, Z4, cmap=cmap, vmin=0,vmax=1)
plt.colorbar(img)
ax4.tick_params(axis='both', which='major', labelsize=14)

In [ ]:
fig.subplots_adjust(top=1)
cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.05])
clb=fig.colorbar(img, cax=cbar_ax, orientation='horizontal')#, label='Factor Intensity (a.u.)')
clb.ax.tick_params(labelsize=14) 
clb.ax.set_title('Factor Intensity (a.u.)',fontsize=14)
cbar_ax.tick_params(labelsize=14)

In [ ]:
ax1.set_axisbelow(False)
ax1.grid(color='gray', linestyle='dashed',linewidth=2)
ax1.set_rlabel_position(0)


ax2.set_axisbelow(False)
ax2.grid(color='gray', linestyle='dashed',linewidth=2)
ax2.set_rlabel_position(0)


ax3.set_axisbelow(False)
ax3.grid(color='gray', linestyle='dashed',linewidth=2)
ax3.set_rlabel_position(0)

ax4.set_axisbelow(False)
ax4.grid(color='gray', linestyle='dashed',linewidth=2)
ax4.set_rlabel_position(0)
plt.rcParams["font.weight"] = "bold"


In [ ]:
plt.tight_layout()
fig.savefig("windrose_4fac_MOD-00400.png", dpi=350)